In [1]:
from glob import glob
from datasets import Dataset
import numpy as np
import pandas as pd
from pprint import pprint
import os
import json
import time
from googletrans import Translator

In [20]:
paths = glob("./data_*.json") + glob("../failed_job_details/data_*.json")
print(len(paths))
ds = []
for path in paths:
    with open(path, "r") as f:
        ds.extend(json.load(f))
ds = Dataset.from_list(ds)
ds = ds.filter(lambda x: x["error"] == False)

917


Filter:   0%|          | 0/9170 [00:00<?, ? examples/s]

In [22]:
translator = Translator()

def translate_text(text, sleep_time=2):
    if text in [""] or text is None:
        return ""
    while True:
        try:
            return translator.translate(text, dest="vi").text
        except Exception() as e:
            print("Requests error")
            time.sleep(sleep_time)

In [23]:
def mapper(x):
    temp = x["company_info"].split("\n")
    company_address, loai_hinh, quy_mo, num_followers = None, None, None, None
    for t in temp:
        if "Địa điểm" in t:
            company_address = t[len("Địa điểm") :].strip()
        elif "Loại hình hoạt động:" in t:
            loai_hinh = t[len("Loại hình hoạt động:") :].strip()
        elif "Qui mô công ty:" in t:
            quy_mo = t[len("Qui mô công ty:") :].strip()
        elif "followers" in t:
            num_followers = t[: t.index("followers")].strip()
            if num_followers == "":
                num_followers = None
            else:
                num_followers = num_followers.replace(",", "").replace(".", "")
                num_followers = int(num_followers)
    x["Ngành nghề"] = [t.strip() for t in x["Ngành nghề"].split(",")]
    experience = x["Kinh nghiệm"]
    min_exp, max_exp = 0, 30
    if experience is not None:
        if "-" in experience:
            words = experience.split(" ")
            min_exp = words[0]
            max_exp = words[2]
        elif "Trên" in experience:
            min_exp = experience.split(" ")[1]
        try:
            min_exp = int(min_exp)
            max_exp = int(max_exp)
        except:
            min_exp, max_exp = 0, 30
    return {
        **x,
        "dia_chi_cong_ty": company_address,
        "loai_hinh_hoat_dong": loai_hinh,
        "quy_mo_cong_ty": quy_mo,
        "num_followers": num_followers,
        "min_exp": min_exp,
        "max_exp": max_exp,
        "Hình thức": [t.strip() for t in x["Hình thức"].split(",")],
    }


ds = ds.map(mapper).select_columns(
    [
        "Ngành nghề",
        "Cấp bậc",
        "Lương",
        "vi_tri_viec",
        "min_exp",
        "max_exp",
        "Địa điểm",
        "ten_cong_ty",
        "dia_chi_cong_ty",
        "loai_hinh_hoat_dong",
        "Hình thức",
        "quy_mo_cong_ty",
        "Ngày cập nhật",
        "Hết hạn nộp",
        "num_followers",
    ]
)
ds = ds.rename_columns({
    'Ngành nghề': 'nganh_nghe',
    'Cấp bậc': 'cap_bac',
    'Lương': 'luong',
    'Địa điểm': 'dia_diem_lam_viec',
    'Hình thức': 'hinh_thuc',
    'Ngày cập nhật': 'ngay_cap_nhat',
    'Hết hạn nộp': 'het_han_nop',
})
def mapper(x):
    x['vi_tri_viec'] = translate_text(x['vi_tri_viec'])
    return x
ds = ds.map(mapper)
ds.to_json("../final/final_dataset.jsonl", orient="records", lines=True, force_ascii=False)
ds

Map:   0%|          | 0/7184 [00:00<?, ? examples/s]

Parameter 'function'=<function mapper at 0x14c247ac0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/7184 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Dataset({
    features: ['vi_tri_viec', 'ten_cong_ty', 'dia_diem_lam_viec', 'ngay_cap_nhat', 'nganh_nghe', 'hinh_thuc', 'luong', 'cap_bac', 'het_han_nop', 'dia_chi_cong_ty', 'loai_hinh_hoat_dong', 'quy_mo_cong_ty', 'num_followers', 'min_exp', 'max_exp'],
    num_rows: 7184
})

In [24]:
ds.to_json(
    "../final/final_dataset.jsonl", orient="records", lines=True, force_ascii=False
)

Creating json from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

4364607

In [25]:
df = ds.to_pandas()
df[~df['luong'].isin(['Cạnh tranh', 'Thỏa thuận'])].shape

(4274, 15)

In [787]:
df['cap_bac'].value_counts()

Nhân viên                   3452
Trưởng nhóm / Giám sát       500
Quản lý                      499
Giám đốc                      91
Sinh viên/ Thực tập sinh      69
Mới tốt nghiệp                48
Phó Giám đốc                  19
Tổng giám đốc                  1
Name: cap_bac, dtype: int64

In [788]:
df['ten_cong_ty'].value_counts()

Ngân Hàng TMCP Việt Nam Thịnh Vượng - VPBANK     101
Ngân Hàng TMCP Quân Đội                           82
Ngân Hàng TMCP Sài Gòn - Hà Nội ( SHB )           58
LPBank                                            57
Công Ty CP Giáo Dục Đại Dương                     55
                                                ... 
Công ty Cổ phần Cổ phần May Minh Anh Thọ Xuân      1
PeopleWise Vietnam                                 1
Công ty TNHH Luki Việt Nam                         1
H. Nicholas & Co Joint Stock Company               1
PG Group - Công ty Cổ phần PG Ads                  1
Name: ten_cong_ty, Length: 1627, dtype: int64

In [789]:
df['loai_hinh_hoat_dong'].value_counts()

Cổ phần                2359
Trách nhiệm hữu hạn    1289
100% vốn nước ngoài     470
Công ty đa quốc gia     105
Cá nhân                  76
Liên doanh               47
Nhà nước                  8
Name: loai_hinh_hoat_dong, dtype: int64

In [790]:
df['hinh_thuc'].explode().value_counts()

Nhân viên chính thức    4579
Thực tập                  77
Bán thời gian             58
Thời vụ/ Nghề tự do       46
Name: hinh_thuc, dtype: int64